In [1]:
from __future__ import print_function, division

In [2]:
from future.utils import iteritems

In [3]:
from builtins import range

In [4]:
import nltk

In [5]:
import numpy as np

In [6]:
from nltk.stem import WordNetLemmatizer

In [7]:
from sklearn.linear_model import LogisticRegression

In [57]:
from bs4 import BeautifulSoup


In [9]:
wordnet_lemmatizer = WordNetLemmatizer()

In [10]:
# Onix Text Retrieval Toolkit provides around 429 Stopwords 
# usually considered as what usually an user searches in a review

In [11]:
# The dataset for stopwords can be downloaded from http://www.lextek.com/manuals/onix/stopwords1.html

In [13]:
stopwords = set(w.rstrip() for w in open('/users/GP/PycharmProjects/Python/Electronics/stopwords.txt'))

In [14]:
# The positive reviews, negative reviews, and unlabled reviews
# can be downloaded from the following Multi-Domain Sentiment Dataset
# http://www.cs.jhu.edu/~mdredze/datasets/sentiment/index2.html

In [59]:
positive_reviews = BeautifulSoup(open('/users/GP/PycharmProjects/Python/Electronics/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [17]:
negative_reviews = BeautifulSoup(open('/users/GP/PycharmProjects/Python/Electronics/negative.review').read())

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [18]:
negative_reviews = negative_reviews.findAll('review_text')

In [19]:
# Based on the dataset, there are more positive reviews found than negative reviews


In [20]:
diff = len(positive_reviews) - len(negative_reviews)

In [21]:
idxs = np.random.choice(len(negative_reviews), size=diff)

In [22]:
extra = [negative_reviews[i] for i in idxs]

In [23]:
negative_reviews += extra

In [25]:

def my_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens


In [27]:
word_index_map = {}
current_index = 0
positive_tokenized = []
negative_tokenized = []

for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [28]:
# creating input matrices

In [29]:
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1) # The last element is for the label
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum() # Perform normalization before setting the label
    x[-1] = label
    return x


In [30]:
N = len(positive_tokenized) + len(negative_tokenized)

In [31]:
data = np.zeros((N, len(word_index_map) + 1))

In [32]:
i = 0

In [33]:
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

In [34]:
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [35]:
# Perform random shuffle and create the data split for train and test

In [36]:
np.random.shuffle(data)

In [37]:
K = data[:,:-1]

In [38]:
Z = data[:,-1]

In [39]:
# Test the last 100 rows

In [41]:
Ktrain = K[:-100,]
Ztrain = Z[:-100,]
Ktest = K[-100:,]
Ztest = Z[-100:,]

In [42]:
model = LogisticRegression()
model.fit(Ktrain, Ztrain)
print("Machine learning classification rate:", model.score(Ktest, Ztest))

Machine learning classification rate: 0.7


In [43]:
# Set different threshold values by looking at the weight of each word

In [44]:
threshold = 0.5
for word, index in iteritems(word_index_map):
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

unit -0.6654115536910598
bad -0.7530980513734542
cable 0.7327088936910008
time -0.655438206274543
've 0.6890309629141371
month -0.7354160723459667
pro 0.5110164469382066
sound 1.0284167296485074
lot 0.7303453599700148
you 0.9890614268534846
n't -2.076753643831598
easy 1.6668537424052763
quality 1.5194027377291928
company -0.5260024382677113
item -0.9109250430895411
wa -1.6313229127613524
perfect 0.986642999168318
fast 0.8901264571515678
ha 0.6843005059287284
price 2.7068565591464564
value 0.567320022960696
money -0.9689483718157904
memory 0.9891753058871854
picture 0.5836717714691185
buy -0.8634162155707061
bit 0.6022461095877204
happy 0.6155667232771299
pretty 0.7579533047223903
doe -1.2796512008978298
highly 1.016011628872127
recommend 0.6695073553233495
customer -0.6553943268222613
support -0.9050893406388059
little 0.9707521309468207
returned -0.7621062577476018
excellent 1.3353095024737722
love 1.2161006438692832
home 0.5941637789618377
week -0.7337195544879093
using 0.60700280262